<a href="https://colab.research.google.com/github/Aleksa1-tech/Hello-world/blob/master/Copy_of_Exercise_3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![PALS0039 Logo](https://www.phon.ucl.ac.uk/courses/pals0039/images/pals0039logo.png)](https://www.phon.ucl.ac.uk/courses/pals0039/)

#Exercise 3.3

In this exercise we train a network to classify vowels from formant data.

(a) Load in vowels data set and prepare for learning. Run the code then add comments to explain how it works.

In [ ]:
# Import the necessary modules/libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%tensorflow_version 2.x
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical

# Import the data as csv file
df=pd.read_csv("https://www.phon.ucl.ac.uk/courses/pals0039/data/vowels.csv")

# See the top rows of the data uploaded
df.head()

---
(b) Convert SEX column to a number. Run the code then add comments to explain how it works.

In [ ]:
# Makes sex a categorical variable (instead of string)
df["SEX"]=df.SEX.astype('category')
# convert SEX column to the numerical codes
df["SEX"]=df.SEX.cat.codes

---
(c) Copy all numbers into numpy array and standardise. Run the code then add comments to explain how it works.

In [ ]:
#print(X)
print(df.iloc[:,3:7])
X.shape[1]

In [ ]:
# Choose the necessary columns 3 to 6 for all rows
X=np.array(df.iloc[:,3:7])
print(X)

# Standardise by substructing mean and dividing by the standard deviation
for i in range(X.shape[1]):
  X[:,i]=(X[:,i]-np.mean(X[:,i]))/np.std(X[:,i])
print(X)


---
(d) Convert vowels to one hot coding. Run the code then add comments to explain how it works.

In [ ]:
#Convert vowel from strings to categories
df["VOWEL"]=df.VOWEL.astype('category')
# convert vowels to the numerical categories and print first ten of them
vowels=df.VOWEL.cat.codes
print(vowels[:10])
#Convert numerical code to one-hot coding
vowelcode = to_categorical(vowels)
print(vowelcode[:10])
# record number of vowels in a dataset 
nvowel=vowelcode.shape[1]
print(nvowel)

---
(e) Divide the data into train and test. Run the code then add comments to explain how it works.

In [ ]:
print(X)
X.shape[1]

In [ ]:
# select 80% of the data for training and 20% for testing
ntrain=int(0.8*X.shape[0])
# get a random permutation of the samples
indices = np.random.permutation(X.shape[0])
# select samples to use for training / these are the main data for testing and training
Xtrain,Xtest = X[indices[:ntrain],:], X[indices[ntrain:],:]
# select samples to use for testing / these are basically labes for training and testing
ytrain,ytest = vowelcode[indices[:ntrain],:], vowelcode[indices[ntrain:],:]


---
(f) Build a classifier network with configurable number of inputs and outputs. Run the code then add comments to explain how it works.

In [ ]:
# define a function for a model
def build_model(numinput,numhidden,numoutput):
  # define the type of a ML model
  model=Sequential()
  # add a first layer defining s number of hidden layers and numinput units
  model.add(Dense(numhidden,activation='sigmoid',input_shape=(numinput,),name="hidden_layer"))
  # add an output layer defining the number of final outputs - build as softmax output
  model.add(Dense(numoutput,activation='softmax',name="output_layer"))
  # define the learning rate and momentum / suitable optimizer
  sgd=SGD(learning_rate=0.1, momentum=0.9)
  # compile the model defining the loss function and optimisation alghorithm
  model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=["accuracy"])
  return model

---
(g) Train the network. Run the code then add comments to explain how it works.

In [ ]:
Xtrain.shape[1]

In [ ]:
# build a model with 4 inputs, 32 hidden layers and the number of vowels equal to a given number
model=build_model(Xtrain.shape[1],32,nvowel);
# fit the model based on the training data and validate it on testing data, returning history
hist=model.fit(Xtrain,ytrain,validation_data=(Xtest,ytest),epochs=100,batch_size=16,verbose=0)
# evaluate the performance of th model on the trainig data
print(model.evaluate(Xtrain,ytrain,verbose=0))

---
(h) Plot training curves. Run the code then add comments to explain how it works.

In [ ]:
# define the figure size
plt.figure(figsize=(10,4))
# plotting next to each other, define the first subplot
plt.subplot(1,2,1)
# plot loss over trainig batches for training loss and validation loss
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(["loss","val_loss"])
# plotting next to each other define the second subplot
plt.subplot(1,2,2)
# plot accuracies over training batches for training and validation
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(["accuracy","val_accuracy"])
plt.show()

---
(i) Calculate performance on test data. Run the code then add comments to explain how it works.

In [ ]:
# predict the vowels for the testing data by ML / predict the vowel codes
vowelcodepred=model.predict(Xtest)
print(vowelcodepred)
# state the real values of vowels / predict the true vowels from the test labels
voweltrue=np.argmax(ytest,axis=1)
print(voweltrue)
# predict vowels takinf the ones with maximum probability as found by the model / get the most probable values in each row
vowelpred=np.argmax(vowelcodepred,axis=1)
print(vowelpred)
# get the total number of vowels predicted / in the test data
total=ytest.shape[0]
# get the percentage correct of the vowels predicted
correct=np.sum(voweltrue==vowelpred)
print("Correct %d/%d (%.1f%%)" % (correct,total,100.0*correct/total))

---
(j) Experiment with changing the number of hidden units, the number of training epochs and the optimizer parameters to see their effect on test set performance.